In [1]:
import pandas as pd
import requests
import folium
import geopandas as gpd
import shapely
from shapely.geometry import Polygon

This notebook loads the FSIS data, filters it for large poultry processors, calls the Mapbox API for driving distance isochrones, and pickles it so it can be directly loaded into other notebooks.

In [2]:
PICKLE_PATH = "../data/pickled-sheepLamb.pickle"
FSIS_DATA = "../data/fsis-processors-with-location.csv"

In [3]:
df = pd.read_csv(FSIS_DATA)
df.head()

,Unnamed: 0,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude
0,0,M267,4802,JBS,JBS Tolleson Inc.,AZ,Large,Beef,5.0,4.0,"651 S. 91st Ave, Tolleson, AZ 85353",33.441660,-112.252559
1,1,M354,5117,Cargill,Cargill Meat Solutions,CA,Large,Beef,5.0,4.0,"3115 S. Fig Ave., Fresno, CA 93706",36.689651,-119.801950
2,2,M6063A,5289,Central Valley Meat Company,"Central Valley Meat Co., Inc.",CA,Large,Beef,5.0,4.0,"10431 8 3/4 Ave., Hanford, CA 93230",36.321273,-119.612222
3,3,M783 + P783 + V783,5144,Central Valley Meat Company,Harris Ranch Beef Company,CA,Large,Beef,5.0,4.0,"16277 S McCall Ave., Selma, CA 93662",36.499212,-119.614553
4,4,M21488 + V21488,8349,Independent,OWB Packers LLC,CA,Large,Beef,4.0,4.0,"57 East Shank Road, Brawley, CA 92227",33.000969,-115.521786


In [4]:
len(df)

1149

In [5]:
# creating DataFrame restricted to Sheep/Lamb
df_sheepLamb = df[df['Animals Processed'].str.contains('Sheep/Lamb') ]
len(df_sheepLamb)

531

In [6]:
df_sheepLamb.Size.value_counts()

Size
Very Small    432
Small          99
Name: count, dtype: int64

In [7]:
# looking at small sheep/lamb
df_small_sheepLamb = df_sheepLamb.loc[df_sheepLamb.Size == "Small"]
df_small_sheepLamb.head()

,Unnamed: 0,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude
405,405,M13149 + P13149 + V13149,3743,Independent,Krehbiels Specialty Meats Inc,KS,Small,"Beef,Pork,Chicken,Turkey,Sheep/Lamb",3.0,3.0,"1636 Mohawk Rd., McPherson, KS 67460",38.409122,-97.624004
408,408,M2800 + P2800,5179,Independent,Superior Farms,CA,Small,Sheep/Lamb,4.0,4.0,"7390 Rio Dixon Rd, Dixon, CA 95620",38.420077,-121.822343
409,409,M47194,6165200,Independent,Colorado Lamb Processors,CO,Small,Sheep/Lamb,1.0,4.0,"1625 Agripark Road, Brush, CO 80723",40.282441,-103.617686
410,410,M5883 + P5883,4898,Independent,Mountain Meadows Lamb Corporation,CO,Small,Sheep/Lamb,4.0,4.0,"4900 Clarkson Street, Denver, CO 80216",39.785791,-104.976580
411,411,M10252,7198,Independent,Berry & Sons Rababeh Isl Slau,MI,Small,Sheep/Lamb,3.0,3.0,"2496 Orleans St., Detroit, MI 48207",42.347092,-83.036872


In [8]:
len(df_small_sheepLamb)

99

### Save Isochrones to Dataframe

In [13]:
# mapbox constants
MAPBOX_TOKEN = "pk.eyJ1IjoidG9kZG5pZWYiLCJhIjoiY2xncGpzbmhhMTBwdzNnbXJjNWlzaTY2aCJ9.UhUELBA2iNIDsTN9YESsIw"
ENDPOINT = "https://api.mapbox.com/isochrone/v1/mapbox/driving/"
DRIVING_DISTANCE = str(int(60 * 1609.34)) # 60 miles in meters: 90 percent of all birds were produced on farms within 60 miles of the plant, according to 2011 ARMS data

In [14]:
isochrones = []
for index, row in df_small_sheepLamb.iterrows():
    lat = str(row['latitude'])
    lng = str(row['longitude'])

    # add driving radius isochrone to map
    url = ENDPOINT + lng + "," + lat + "?" + "contours_meters=" + DRIVING_DISTANCE + "&access_token=" + MAPBOX_TOKEN
    response = requests.get(url)
    isochrone = Polygon(response.json()['features'][0]['geometry']['coordinates'])
    isochrones.append(isochrone)

In [15]:
df_small_sheepLamb = df_small_sheepLamb.copy()
df_small_sheepLamb["Isochrone"] = isochrones

In [16]:
df_small_sheepLamb.head()

,Unnamed: 0,EstNumber,EstID,Parent Corporation,Establishment Name,State,Size,Animals Processed,Processed\nVolume\nCategory,Slaughter\nVolume\nCategory,Full Address,latitude,longitude,Isochrone
405,405,M13149 + P13149 + V13149,3743,Independent,Krehbiels Specialty Meats Inc,KS,Small,"Beef,Pork,Chicken,Turkey,Sheep/Lamb",3.0,3.0,"1636 Mohawk Rd., McPherson, KS 67460",38.409122,-97.624004,"POLYGON ((-97.669003 39.239398, -97.670536 39...."
408,408,M2800 + P2800,5179,Independent,Superior Farms,CA,Small,Sheep/Lamb,4.0,4.0,"7390 Rio Dixon Rd, Dixon, CA 95620",38.420077,-121.822343,"POLYGON ((-121.675343 39.197236, -121.678343 3..."
409,409,M47194,6165200,Independent,Colorado Lamb Processors,CO,Small,Sheep/Lamb,1.0,4.0,"1625 Agripark Road, Brush, CO 80723",40.282441,-103.617686,"POLYGON ((-103.665686 41.053657, -103.666876 4..."
410,410,M5883 + P5883,4898,Independent,Mountain Meadows Lamb Corporation,CO,Small,Sheep/Lamb,4.0,4.0,"4900 Clarkson Street, Denver, CO 80216",39.785791,-104.976580,"POLYGON ((-105.00058 40.64406, -105.002501 40...."
411,411,M10252,7198,Independent,Berry & Sons Rababeh Isl Slau,MI,Small,Sheep/Lamb,3.0,3.0,"2496 Orleans St., Detroit, MI 48207",42.347092,-83.036872,"POLYGON ((-83.072872 43.10869, -83.075055 43.0..."


In [17]:
df_small_sheepLamb.to_pickle(PICKLE_PATH)